# LSTM Series

** - Matlab initialization strategy **

** - Train and Dev different loss functions **

** - Adam Optimizer **

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import math

import scipy.io as sio
import numpy as np
import h5py
import tensorflow as tf
import sys

import time

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"


from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, LSTM, GRU, Dense, Dropout, Bidirectional
from keras.models import Sequential
from keras.layers import Dense, Activation, TimeDistributed
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [2]:
# Configuration to control GPU use
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.33
sess = tf.InteractiveSession(config=config)

### Read parameters from .mat files
save in Opt object

In [2]:
Data_VERSION = "_e11v1"
Code_VERSION = "_e11v1"

In [3]:
DNN_DATA_FILE = "./dnn_models/DNN_datas"+ Data_VERSION+".mat"
# DNN_DATA_FILE = "./dnn_models/BR2_DNN_datas.mat"

DNN_MODEL_FILE = "./dnn_models/DNN_params"+ Data_VERSION+".mat"
# DNN_MODEL_FILE = "./dnn_models/BR2_DNN_params.mat"

DNN_NET_FILE = "./dnn_models/DNN_net"+ Code_VERSION+".mat"

# ModelFN_FILE = "./dnn_models/dnncirm.noiseSSN_05.mat"

Files saved by Matlab reading class. Data, Parameters are read. 

In [4]:
class Opts:
    opts_dict = dict()

    def __init__(self, FILE, FILE_DATA):
        
        # Basic parameters
        with h5py.File(FILE, 'r') as f:
            key_list = list(f.keys())
            print('Opt key:',key_list)

            for k, v in f['opts'].items():

                print('key:', k)

                if k == 'ARMA_order':
                    self.ARMA_order = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.ARMA_order
                elif k == 'ada_grad_eps':
                    self.ada_grad_eps = np.array(v)[0][0]
                    self.opts_dict[k] = self.ada_grad_eps
                elif k == 'ada_sgd_scale':
                    self.ada_sgd_scale = np.array(v)[0][0]
                    self.opts_dict[k] = self.ada_sgd_scale
                elif k == 'change_momentum_point':
                    self.change_momentum_point = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.change_momentum_point
                elif k == 'cost_function':
                    self.cost_function = ""
                    for c in np.array(v):
                        self.cost_function += chr(c[0])

                    self.opts_dict[k] = self.cost_function

                elif k == 'cv_interval':
                    self.cv_interval = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.cv_interval
                elif k == 'dim_input':
                    self.dim_input = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.dim_input
                elif k == 'dim_output':
                    self.dim_output = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.dim_output
                elif k == 'drop_ratio':
                    self.drop_ratio = np.array(v)[0][0]
                    self.opts_dict[k] = self.drop_ratio
                elif k == 'eval_on_gpu':
                    self.eval_on_gpu = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.eval_on_gpu
                elif k == 'final_momentum':
                    self.final_momentum = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.final_momentum
                elif k == 'hid_struct':
                    self.hid_struct = np.array(v)
                    self.opts_dict[k] = self.hid_struct
                elif k == 'initial_momentum':
                    self.initial_momentum = np.array(v)[0][0]
                    self.opts_dict[k] = self.initial_momentum
                elif k == 'isDropout':
                    self.isDropout = 0
                    self.opts_dict[k] = self.isDropout
                elif k == 'isDropoutInput':
                    self.isDropoutInput = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.isDropoutInput
                elif k == 'isGPU':
                    self.isGPU = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.isGPU
                elif k == 'isNormalize':
                    self.isNormalize = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.isNormalize
                elif k == 'isPretrain':
                    self.isPretrain = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.isPretrain
                elif k == 'learner':
                    self.learner = ""
                    for c in np.array(v):
                        self.learner += chr(c[0])

                    self.opts_dict[k] = self.learner

                elif k == 'net_struct':
                    self.net_struct = np.array(v)
                    for n_s in np.array(v):
                        print('Opts Net Stuct:',n_s[0])

                    self.opts_dict[k] = self.net_struct
                elif k == 'rbm_batch_size':
                    self.rbm_batch_size = int(np.array(v)[0][0])
                    print("self.rbm_batch_size:",self.rbm_batch_size)
                    # self.opts_dict[k] = self.rbm_batch_size
                elif k == 'rbm_learn_rate_binary':
                    self.rbm_learn_rate_binary = np.array(v)
                    # self.opts_dict[k] = self.rbm_learn_rate_binary
                elif k == 'rbm_learn_rate_real':
                    self.rbm_learn_rate_real = int(np.array(v)[0][0])
                    # self.opts_dict[k] = self.rbm_learn_rate_real
                elif k == 'rbm_max_epoch':
                    self.rbm_max_epoch = int(np.array(v)[0][0])
                    # self.opts_dict[k] = self.rbm_max_epoch
                elif k == 'save_on_fly':
                    self.save_on_fly = int(np.array(v)[0][0])
                    # self.opts_dict[k] = self.save_on_fly
                elif k == 'sgd_batch_size':
                    self.sgd_batch_size = int(np.array(v)[0][0])
                    print("self.sgd_batch_size:",self.sgd_batch_size)
#                     self.sgd_batch_size = 1024//2 # BATCH_SIZE for training net
                    # self.opts_dict[k] = self.sgd_batch_size
                elif k == 'sgd_learn_rate':
                    self.sgd_learn_rate = np.array(v)
                    # self.opts_dict[k] = self.sgd_learn_rate
                elif k == 'sgd_max_epoch':
                    self.sgd_max_epoch = int(np.array(v)[0][0])
                    # self.opts_dict[k] = self.sgd_max_epoch
                elif k == 'split_tanh1_c1':
                    self.split_tanh1_c1 = int(np.array(v)[0][0])
                    # self.opts_dict[k] = self.split_tanh1_c1
                elif k == 'split_tanh1_c2':
                    self.split_tanh1_c2 = int(np.array(v)[0][0])
                    # self.opts_dict[k] = self.split_tanh1_c2
                elif k == 'unit_type_hidden':
                    self.unit_type_hidden = ""
                    for c in np.array(v):
                        self.unit_type_hidden += chr(c[0])

                elif k == 'unit_type_output':
                    self.unit_type_output = ""
                    for c in np.array(v):
                        self.unit_type_output += chr(c[0])

        # Training and Dev Data 
        with h5py.File(FILE_DATA, 'r') as f:
            print('Opts h5py keys:', list(f.keys()))
            for k, v in f.items():

                if k == 'trData':
                    print("trData.shape: ", v.shape)
                    self.trData = np.transpose(np.array(v))
#                     print("trData-> mean:", np.mean(self.trData),", var:",np.var(self.trData), ", std:",np.std(self.trData))
                elif k == 'trLabel_i':
                    print("trLabel_i.shape: ", v.shape)
                    self.trLabel_i = np.transpose(np.array(v))
                    print("trLabel_i-> mean:", np.mean(self.trLabel_i), ", var:", np.var(self.trLabel_i), ", std:",
                          np.std(self.trLabel_i), ", range:", (np.amin(self.trLabel_i),np.amax(self.trLabel_i)))
                elif k == 'trLabel_r':
                    print("trLabel_r.shape: ", v.shape)
                    self.trLabel_r = np.transpose(np.array(v))
                    print("trLabel_r-> mean:", np.mean(self.trLabel_r), ", var:", np.var(self.trLabel_r), ", std:",
                          np.std(self.trLabel_r), ", range:", (np.amin(self.trLabel_r),np.amax(self.trLabel_r)))
                elif k == 'trNumframes':
                    print("trNumframes.shape: ", v.shape)
                    self.trNumframes = np.transpose(np.array(v))
                    
                elif k == 'cvData':
                    print("cvData.shape: ", v.shape)
                    self.cvData = np.transpose(np.array(v))
#                     print("cvData-> mean:", np.mean(self.cvData), ", var:", np.var(self.cvData), ", std:",
#                           np.std(self.cvData))
                elif k == 'cvLabel_i':
                    print("cvLabel_i.shape: ", v.shape)
                    self.cvLabel_i = np.transpose(np.array(v))
                    print("cvLabel_i-> mean:", np.mean(self.cvLabel_i), ", var:", np.var(self.cvLabel_i), ", std:",
                          np.std(self.cvLabel_i), ", range:", (np.amin(self.cvLabel_i),np.amax(self.cvLabel_i)))
                elif k == 'cvLabel_r':
                    print("cvLabel_r.shape: ", v.shape)
                    self.cvLabel_r = np.transpose(np.array(v))
                    print("cvLabel_r-> mean:", np.mean(self.cvLabel_r), ", var:", np.var(self.cvLabel_r), ", std:",
                          np.std(self.cvLabel_r), ", range:", (np.amin(self.cvLabel_r),np.amax(self.cvLabel_r)))
                elif k == 'cvNumframes':
                    print("cvNumframes.shape: ", v.shape)
                    self.cvNumframes = np.transpose(np.array(v))

            self.trLabel = np.concatenate((self.trLabel_r, self.trLabel_i), axis=1)
            self.cvLabel = np.concatenate((self.cvLabel_r, self.cvLabel_i), axis=1)


            
#     def prepare3D_data(self, isTRAINING = True, typ='keras', maxlen=None):
#         # TRAIN: total_num_samples = self.trData.shape[0] = 195192
#         # DEV: total_num_samples = self.trData.shape[0] = 44961

#         feat_vec_len = 1230
#         out_vec_len = 963
        
#         if isTRAINING:
#             data = self.trData
#             label_r = self.trLabel_r
#             label_i = self.trLabel_i
#             numframes = self.trNumframes
#         else:
#             data = self.cvData
#             label_r = self.cvLabel_r
#             label_i = self.cvLabel_i
#             numframes = self.cvNumframes

#         data3D, label3D_r, label3D_i = [], [], []
#         numframes = np.cumsum(numframes)
        
#         #### WORKING ON IT ...
#         for e, frames in enumerate(numframes):
#             frames= int(frames)
#             pre_frames= int(numframes[e-1])
            
#             d = data[:frames] if len(data3D)==0 else data[pre_frames:frames]
#             r = label_r[:frames] if len(label3D_r)==0 else label_r[pre_frames:frames]
#             i = label_i[:frames] if len(label3D_i)==0 else label_i[pre_frames:frames]
            
#             if typ.lower() == 'keras':
#                 d = d.flatten()
#                 r = r.flatten()
#                 i = i.flatten()
                
#             data3D.append( d )
#             label3D_r.append( r )
#             label3D_i.append( i )
# #             print("{0}: data3D.len={1}, label3D_r.len={2}, label3D_r.len={3}".format(e,len(data3D),len(label3D_r),len(label3D_r)))
# #             print("d:{0}, r:{1}, i:{2}".format(d.shape, r.shape, i.shape))
        
#         if typ.lower() == 'keras':
#             maxlen = None if maxlen is None else maxlen*feat_vec_len
#             data3D = pad_sequences(data3D, maxlen=maxlen, dtype='float', padding='post', truncating='post', value=0.0)
            
#             maxlen = None if maxlen is None else maxlen*out_vec_len
#             label3D_r = pad_sequences(label3D_r, maxlen=maxlen, dtype='float', padding='post', truncating='post', value=0.0)
            
#             maxlen = None if maxlen is None else maxlen*out_vec_len
#             label3D_i = pad_sequences(label3D_i, maxlen=maxlen, dtype='float', padding='post', truncating='post', value=0.0)
            
#             print("# data3D.len={0}, label3D_r.len={1}, label3D_r.len={2}".format(data3D.shape,label3D_r.shape,label3D_r.shape))
            
            
#             data3D = data3D.reshape(e,-1,feat_vec_len)
#             label3D_r = label3D_r.flatten(e,-1,out_vec_len)
#             label3D_i = label3D_i.flatten(e,-1,out_vec_len)
            
#         print("## data3D.len={0}, label3D_r.len={1}, label3D_r.len={2}".format(data3D.shape,label3D_r.shape,label3D_r.shape))
                
            
#         return data3D, label3D_r, label3D_i
    
    
    def prepare3D_data2(self, isTRAINING = True, typ='keras', maxlen=None):
        # TRAIN: total_num_samples = self.trData.shape[0] = 195192
        # DEV: total_num_samples = self.trData.shape[0] = 44961

        feat_vec_len = 1230
        out_vec_len = 963
        
        if isTRAINING:
            data = self.trData
            label_r = self.trLabel_r
            label_i = self.trLabel_i
            numframes = self.trNumframes
        else:
            data = self.cvData
            label_r = self.cvLabel_r
            label_i = self.cvLabel_i
            numframes = self.cvNumframes

        data3D, label3D_r, label3D_i = [], [], []
        numframes = np.cumsum(numframes)
        
        #### WORKING ON IT ...
        for e, frames in enumerate(numframes):
            frames= int(frames)
            pre_frames= int(numframes[e-1])
            
            d = data[:frames] if len(data3D)==0 else data[pre_frames:frames]
            r = label_r[:frames] if len(label3D_r)==0 else label_r[pre_frames:frames]
            i = label_i[:frames] if len(label3D_i)==0 else label_i[pre_frames:frames]
            
            if typ.lower() == 'keras':
                frames = d.shape[0]
                d = d.flatten()
                r = r.flatten()
                i = i.flatten()
                
            data3D.append( d )
            label3D_r.append( r )
            label3D_i.append( i )
            print("{0}: data3D.len={1}, label3D_r.len={2}, label3D_r.len={3}".format(e,len(data3D),len(label3D_r),len(label3D_r)))
            print("d:{0}, r:{1}, i:{2}".format(d.shape, r.shape, i.shape))
            return
        
        if typ.lower() == 'keras':
            maxlen = None if maxlen is None else maxlen*feat_vec_len
            data3D = pad_sequences(data3D, maxlen=maxlen, dtype='float', padding='post', truncating='post', value=0.0)
            
            maxlen = None if maxlen is None else maxlen*out_vec_len
            label3D_r = pad_sequences(label3D_r, maxlen=maxlen, dtype='float', padding='post', truncating='post', value=0.0)
            
            maxlen = None if maxlen is None else maxlen*out_vec_len
            label3D_i = pad_sequences(label3D_i, maxlen=maxlen, dtype='float', padding='post', truncating='post', value=0.0)
            
            print("# data3D.len={0}, label3D_r.len={1}, label3D_r.len={2}".format(data3D.shape,label3D_r.shape,label3D_r.shape))
            
            
            data3D = data3D.reshape(e,-1,feat_vec_len)
            label3D_r = label3D_r.flatten(e,-1,out_vec_len)
            label3D_i = label3D_i.flatten(e,-1,out_vec_len)
            
        print("## data3D.len={0}, label3D_r.len={1}, label3D_r.len={2}".format(data3D.shape,label3D_r.shape,label3D_r.shape))
                
            
        return data3D, label3D_r, label3D_i

    
    def prepare3D_list(self, isTRAINING = True):
        # TRAIN: total_num_samples = self.trData.shape[0] = 195192
        # DEV: total_num_samples = self.trData.shape[0] = 44961

        feat_vec_len = 1230
        out_vec_len = 963
        
        if isTRAINING:
            data = self.trData
            label_r = self.trLabel_r
            label_i = self.trLabel_i
            numframes = self.trNumframes
        else:
            data = self.cvData
            label_r = self.cvLabel_r
            label_i = self.cvLabel_i
            numframes = self.cvNumframes

        data3D, label3D_r, label3D_i = [], [], []
        numframes = np.cumsum(numframes)
        
        for e, frames in enumerate(numframes):
            frames= int(frames)
            pre_frames= int(numframes[e-1])
            
            d = data[:frames] if len(data3D)==0 else data[pre_frames:frames]
            r = label_r[:frames] if len(label3D_r)==0 else label_r[pre_frames:frames]
            i = label_i[:frames] if len(label3D_i)==0 else label_i[pre_frames:frames]
            
            data3D.append( d )
            label3D_r.append( r )
            label3D_i.append( i )

        return data3D, label3D_r, label3D_i

    def ready_batchID(self, total_num_samples, batch_size):
        # TRAIN: total_num_samples = self.trData.shape[0] = 195192
        # DEV: total_num_samples = self.trData.shape[0] = 44961

        batchID = []
        num_batch = math.ceil(total_num_samples/batch_size)

        for b in range( int(num_batch) ):
            s = b*batch_size
            e = (b+1)*batch_size -1

            if e >= total_num_samples:
                e = total_num_samples - 1

            batchID.append((s,e))

        return np.array(batchID,ndmin=2)


    def suffle_data(self, total_num_samples):
        # TRAIN: total_num_samples = self.trData.shape[0] = 195192
        # DEV: total_num_samples = self.trData.shape[0] = 44961

        return  np.random.permutation(total_num_samples)


    def next_batch(self, total_num_samples, batch_size, isTrainCycle=True):
        # TRAIN: total_num_samples = self.trData.shape[0] = 195192
        # DEV: total_num_samples = self.trData.shape[0] = 44961

        batchID = self.ready_batchID(total_num_samples, batch_size)
        seq = self.suffle_data(total_num_samples)

        for batch in range(batchID.shape[0]):
            if isTrainCycle:
                x = opts.trData[ seq[batchID[batch][0]:batchID[batch][1] ] ]
                y = opts.trLabel[ seq[batchID[batch][0]:batchID[batch][1] ] ]

            else:
                x = opts.cvData[seq[batchID[batch][0]:batchID[batch][1]]]
                y = opts.cvLabel[seq[batchID[batch][0]:batchID[batch][1]]]

            # print('Next Batch', x.shape, y.shape)
            yield [x, y]
            
            
    # not use (OLD version)
    def next_batch_2(self, batch_size, isTrainCycle=True):
        if isTrainCycle:
            # selected_indics = np.random.choice(10, size=batch_size, replace=False)
            selected_indics = np.random.randint(self.trData.shape[1], size=batch_size)
        else:
            # selected_indics = np.random.randint(7, size= batch_size)
            selected_indics = np.random.randint(44961, size=batch_size)

        # print ("selected_indics: ",selected_indics)
        if isTrainCycle:
            x = opts.trData[selected_indics]
            y = np.concatenate((opts.trLabel_r, opts.trLabel_i), axis=1)[selected_indics]

        else:
            x = opts.cvData[selected_indics]
            y = np.concatenate((opts.cvLabel_r, opts.cvLabel_i), axis=1)[selected_indics]

        # print('Next Batch', x.shape, y.shape)
        return [x, y]


In [5]:
opts = Opts(DNN_MODEL_FILE, DNN_DATA_FILE)

Opt key: ['#refs#', 'opts']
key: ARMA_order
key: ada_grad_eps
key: ada_sgd_scale
key: change_momentum_point
key: cost_function
key: cv_interval
key: dim_input
key: dim_output
key: drop_ratio
key: eval_on_gpu
key: final_momentum
key: hid_struct
key: initial_momentum
key: isDropout
key: isDropoutInput
key: isGPU
key: isNormalize
key: isPretrain
key: learner
key: net_struct
Opts Net Stuct: <HDF5 object reference>
Opts Net Stuct: <HDF5 object reference>
Opts Net Stuct: <HDF5 object reference>
Opts Net Stuct: <HDF5 object reference>
Opts Net Stuct: <HDF5 object reference>
key: rbm_batch_size
self.rbm_batch_size: 512
key: rbm_learn_rate_binary
key: rbm_learn_rate_real
key: rbm_max_epoch
key: save_on_fly
key: sgd_batch_size
self.sgd_batch_size: 512
key: sgd_learn_rate
key: sgd_max_epoch
key: split_tanh1_c1
key: split_tanh1_c2
key: tr_mu
key: tr_std
key: unit_type_hidden
key: unit_type_output
Opts h5py keys: ['cvData', 'cvLabel_i', 'cvLabel_r', 'cvNumframes', 'trData', 'trLabel_i', 'trLabel_r'

#### For RNN style 3D batch generation

In [6]:
def next_batch_rnn(opts, batch_size, maxlen=185, isTrainCycle=True):
    # TRAIN: total_num_samples = self.trData.shape[0] = 1951920
    # DEV: total_num_samples = self.trData.shape[0] = 449610

    feat_vec_len = 1230
    out_vec_len = 963

    if isTrainCycle:
        selected_indics = np.arange(len(opts.trNumframes))
        data3D, label3D_r, label3D_i = opts.prepare3D_list(isTRAINING = True)
    else:
        selected_indics = np.arange(len(opts.cvNumframes))
        data3D, label3D_r, label3D_i = opts.prepare3D_list(isTRAINING = False)

    np.random.shuffle(selected_indics)


    if isTrainCycle or not isTrainCycle:
        f = 1
        while (f*batch_size) < len(selected_indics):
            si = selected_indics[((f-1)*batch_size):(f*batch_size)]

            x, y = None, None
            for indx in si:
                d = data3D[indx]
                d = np.concatenate( (d, np.zeros((maxlen-d.shape[0],feat_vec_len))),axis=0 )
                d = np.expand_dims(d, axis=0)
                
                l = np.concatenate((label3D_r[indx], label3D_i[indx]), axis=1)
                l = np.concatenate( (l, np.zeros((maxlen-l.shape[0],out_vec_len*2))),axis=0 )
                l = np.expand_dims(l, axis=0)

                x = d if x is None else np.concatenate( (x,d),axis=0 )
                y = l if y is None else np.concatenate( (y,l),axis=0 )

            f += 1
            yield x,y
            
            
#     else:
#         
#         x, y = None, None
#         for indx in selected_indics:
#             d = data3D[indx]
#             d = np.concatenate( (d, np.zeros((maxlen-d.shape[0],feat_vec_len))),axis=0 )
#             d = np.expand_dims(d, axis=0)

#             l = np.concatenate((label3D_r[indx], label3D_i[indx]), axis=1)
#             l = np.concatenate( (l, np.zeros((maxlen-l.shape[0],out_vec_len*2))),axis=0 )
#             l = np.expand_dims(l, axis=0)

#             x = d if x is None else np.concatenate( (x,d),axis=0 )
#             y = l if y is None else np.concatenate( (y,l),axis=0 )
                
#         return x, y

In [97]:
for e,[x,y] in enumerate(next_batch_rnn(opts,64,isTrainCycle=False)):
    print(x.shape, y.shape)

(64, 185, 1230) (64, 185, 1926)
(64, 185, 1230) (64, 185, 1926)
(64, 185, 1230) (64, 185, 1926)


KeyboardInterrupt: 

In [73]:
xy[0]

TypeError: 'generator' object is not subscriptable

In [ ]:
maxlen=185
isTRAINING = True
typ = 'keras'

feat_vec_len = 1230
out_vec_len = 963

if isTRAINING:
    data = opts.trData
    label_r = opts.trLabel_r
    label_i = opts.trLabel_i
    numframes = opts.trNumframes


data3D, label3D_r, label3D_i = None, None, None
numframes = np.cumsum(numframes)

#### WORKING ON IT ...
for e, frames in enumerate(numframes):
    frames= int(frames)
    pre_frames= int(numframes[e-1])

    # separate frames for each wavs
    d = data[:frames] if data3D is None else data[pre_frames:frames]
    r = label_r[:frames] if label3D_r is None else label_r[pre_frames:frames]
    i = label_i[:frames] if label3D_i is None else label_i[pre_frames:frames]

    # flattend(1-D) feature array
    # Add zeros add the end to make same sized, then again 2D for concat
    if typ.lower() == 'keras':
        frame = d.shape[0]

        d = d.flatten()
        d = np.array( np.concatenate( (d, np.zeros((maxlen-frame)*feat_vec_len)) ), ndmin=2)
        
        r = r.flatten()
        r = np.array( np.concatenate( (r, np.zeros((maxlen-frame)*out_vec_len)) ), ndmin=2)
        
        i = i.flatten()
        i = np.array( np.concatenate( (i, np.zeros((maxlen-frame)*out_vec_len)) ), ndmin=2)


    data3D= d if data3D is None else np.concatenate( (data3D,d),axis=0 )
    label3D_r= r if label3D_r is None else np.concatenate( (label3D_r,r),axis=0 )
    label3D_i= i if label3D_i is None else np.concatenate( (label3D_i,i),axis=0 )
    
    print("{0}: data3D.len={1}, label3D_r.len={2}, label3D_r.len={3}".format(e,len(data3D),len(label3D_r),len(label3D_r)))
#     print("d:{0}, r:{1}, i:{2}".format(d.shape, r.shape, i.shape))
    
#     print("# data3D.len={0}, label3D_r.len={1}, label3D_r.len={2}".format(data3D.shape,label3D_r.shape,label3D_r.shape))

#     if e == 2:
#         break
        

0: data3D.len=1, label3D_r.len=1, label3D_r.len=1
1: data3D.len=2, label3D_r.len=2, label3D_r.len=2
2: data3D.len=3, label3D_r.len=3, label3D_r.len=3
3: data3D.len=4, label3D_r.len=4, label3D_r.len=4
4: data3D.len=5, label3D_r.len=5, label3D_r.len=5
5: data3D.len=6, label3D_r.len=6, label3D_r.len=6
6: data3D.len=7, label3D_r.len=7, label3D_r.len=7
7: data3D.len=8, label3D_r.len=8, label3D_r.len=8
8: data3D.len=9, label3D_r.len=9, label3D_r.len=9
9: data3D.len=10, label3D_r.len=10, label3D_r.len=10
10: data3D.len=11, label3D_r.len=11, label3D_r.len=11
11: data3D.len=12, label3D_r.len=12, label3D_r.len=12
12: data3D.len=13, label3D_r.len=13, label3D_r.len=13
13: data3D.len=14, label3D_r.len=14, label3D_r.len=14
14: data3D.len=15, label3D_r.len=15, label3D_r.len=15
15: data3D.len=16, label3D_r.len=16, label3D_r.len=16
16: data3D.len=17, label3D_r.len=17, label3D_r.len=17
17: data3D.len=18, label3D_r.len=18, label3D_r.len=18
18: data3D.len=19, label3D_r.len=19, label3D_r.len=19
19: data3D.

149: data3D.len=150, label3D_r.len=150, label3D_r.len=150
150: data3D.len=151, label3D_r.len=151, label3D_r.len=151
151: data3D.len=152, label3D_r.len=152, label3D_r.len=152
152: data3D.len=153, label3D_r.len=153, label3D_r.len=153
153: data3D.len=154, label3D_r.len=154, label3D_r.len=154
154: data3D.len=155, label3D_r.len=155, label3D_r.len=155
155: data3D.len=156, label3D_r.len=156, label3D_r.len=156
156: data3D.len=157, label3D_r.len=157, label3D_r.len=157
157: data3D.len=158, label3D_r.len=158, label3D_r.len=158
158: data3D.len=159, label3D_r.len=159, label3D_r.len=159
159: data3D.len=160, label3D_r.len=160, label3D_r.len=160
160: data3D.len=161, label3D_r.len=161, label3D_r.len=161
161: data3D.len=162, label3D_r.len=162, label3D_r.len=162
162: data3D.len=163, label3D_r.len=163, label3D_r.len=163
163: data3D.len=164, label3D_r.len=164, label3D_r.len=164
164: data3D.len=165, label3D_r.len=165, label3D_r.len=165
165: data3D.len=166, label3D_r.len=166, label3D_r.len=166
166: data3D.le

291: data3D.len=292, label3D_r.len=292, label3D_r.len=292
292: data3D.len=293, label3D_r.len=293, label3D_r.len=293
293: data3D.len=294, label3D_r.len=294, label3D_r.len=294
294: data3D.len=295, label3D_r.len=295, label3D_r.len=295
295: data3D.len=296, label3D_r.len=296, label3D_r.len=296
296: data3D.len=297, label3D_r.len=297, label3D_r.len=297
297: data3D.len=298, label3D_r.len=298, label3D_r.len=298
298: data3D.len=299, label3D_r.len=299, label3D_r.len=299
299: data3D.len=300, label3D_r.len=300, label3D_r.len=300
300: data3D.len=301, label3D_r.len=301, label3D_r.len=301
301: data3D.len=302, label3D_r.len=302, label3D_r.len=302
302: data3D.len=303, label3D_r.len=303, label3D_r.len=303
303: data3D.len=304, label3D_r.len=304, label3D_r.len=304
304: data3D.len=305, label3D_r.len=305, label3D_r.len=305
305: data3D.len=306, label3D_r.len=306, label3D_r.len=306
306: data3D.len=307, label3D_r.len=307, label3D_r.len=307
307: data3D.len=308, label3D_r.len=308, label3D_r.len=308
308: data3D.le

433: data3D.len=434, label3D_r.len=434, label3D_r.len=434
434: data3D.len=435, label3D_r.len=435, label3D_r.len=435
435: data3D.len=436, label3D_r.len=436, label3D_r.len=436
436: data3D.len=437, label3D_r.len=437, label3D_r.len=437
437: data3D.len=438, label3D_r.len=438, label3D_r.len=438
438: data3D.len=439, label3D_r.len=439, label3D_r.len=439
439: data3D.len=440, label3D_r.len=440, label3D_r.len=440
440: data3D.len=441, label3D_r.len=441, label3D_r.len=441
441: data3D.len=442, label3D_r.len=442, label3D_r.len=442
442: data3D.len=443, label3D_r.len=443, label3D_r.len=443
443: data3D.len=444, label3D_r.len=444, label3D_r.len=444
444: data3D.len=445, label3D_r.len=445, label3D_r.len=445
445: data3D.len=446, label3D_r.len=446, label3D_r.len=446
446: data3D.len=447, label3D_r.len=447, label3D_r.len=447
447: data3D.len=448, label3D_r.len=448, label3D_r.len=448
448: data3D.len=449, label3D_r.len=449, label3D_r.len=449
449: data3D.len=450, label3D_r.len=450, label3D_r.len=450
450: data3D.le

575: data3D.len=576, label3D_r.len=576, label3D_r.len=576
576: data3D.len=577, label3D_r.len=577, label3D_r.len=577
577: data3D.len=578, label3D_r.len=578, label3D_r.len=578
578: data3D.len=579, label3D_r.len=579, label3D_r.len=579
579: data3D.len=580, label3D_r.len=580, label3D_r.len=580
580: data3D.len=581, label3D_r.len=581, label3D_r.len=581
581: data3D.len=582, label3D_r.len=582, label3D_r.len=582
582: data3D.len=583, label3D_r.len=583, label3D_r.len=583
583: data3D.len=584, label3D_r.len=584, label3D_r.len=584
584: data3D.len=585, label3D_r.len=585, label3D_r.len=585
585: data3D.len=586, label3D_r.len=586, label3D_r.len=586
586: data3D.len=587, label3D_r.len=587, label3D_r.len=587
587: data3D.len=588, label3D_r.len=588, label3D_r.len=588
588: data3D.len=589, label3D_r.len=589, label3D_r.len=589
589: data3D.len=590, label3D_r.len=590, label3D_r.len=590
590: data3D.len=591, label3D_r.len=591, label3D_r.len=591
591: data3D.len=592, label3D_r.len=592, label3D_r.len=592
592: data3D.le

717: data3D.len=718, label3D_r.len=718, label3D_r.len=718
718: data3D.len=719, label3D_r.len=719, label3D_r.len=719
719: data3D.len=720, label3D_r.len=720, label3D_r.len=720
720: data3D.len=721, label3D_r.len=721, label3D_r.len=721
721: data3D.len=722, label3D_r.len=722, label3D_r.len=722
722: data3D.len=723, label3D_r.len=723, label3D_r.len=723
723: data3D.len=724, label3D_r.len=724, label3D_r.len=724
724: data3D.len=725, label3D_r.len=725, label3D_r.len=725
725: data3D.len=726, label3D_r.len=726, label3D_r.len=726
726: data3D.len=727, label3D_r.len=727, label3D_r.len=727
727: data3D.len=728, label3D_r.len=728, label3D_r.len=728
728: data3D.len=729, label3D_r.len=729, label3D_r.len=729
729: data3D.len=730, label3D_r.len=730, label3D_r.len=730
730: data3D.len=731, label3D_r.len=731, label3D_r.len=731
731: data3D.len=732, label3D_r.len=732, label3D_r.len=732
732: data3D.len=733, label3D_r.len=733, label3D_r.len=733
733: data3D.len=734, label3D_r.len=734, label3D_r.len=734
734: data3D.le

859: data3D.len=860, label3D_r.len=860, label3D_r.len=860
860: data3D.len=861, label3D_r.len=861, label3D_r.len=861
861: data3D.len=862, label3D_r.len=862, label3D_r.len=862
862: data3D.len=863, label3D_r.len=863, label3D_r.len=863
863: data3D.len=864, label3D_r.len=864, label3D_r.len=864
864: data3D.len=865, label3D_r.len=865, label3D_r.len=865
865: data3D.len=866, label3D_r.len=866, label3D_r.len=866
866: data3D.len=867, label3D_r.len=867, label3D_r.len=867
867: data3D.len=868, label3D_r.len=868, label3D_r.len=868
868: data3D.len=869, label3D_r.len=869, label3D_r.len=869
869: data3D.len=870, label3D_r.len=870, label3D_r.len=870
870: data3D.len=871, label3D_r.len=871, label3D_r.len=871
871: data3D.len=872, label3D_r.len=872, label3D_r.len=872
872: data3D.len=873, label3D_r.len=873, label3D_r.len=873
873: data3D.len=874, label3D_r.len=874, label3D_r.len=874
874: data3D.len=875, label3D_r.len=875, label3D_r.len=875
875: data3D.len=876, label3D_r.len=876, label3D_r.len=876
876: data3D.le

1001: data3D.len=1002, label3D_r.len=1002, label3D_r.len=1002
1002: data3D.len=1003, label3D_r.len=1003, label3D_r.len=1003
1003: data3D.len=1004, label3D_r.len=1004, label3D_r.len=1004
1004: data3D.len=1005, label3D_r.len=1005, label3D_r.len=1005
1005: data3D.len=1006, label3D_r.len=1006, label3D_r.len=1006
1006: data3D.len=1007, label3D_r.len=1007, label3D_r.len=1007
1007: data3D.len=1008, label3D_r.len=1008, label3D_r.len=1008
1008: data3D.len=1009, label3D_r.len=1009, label3D_r.len=1009
1009: data3D.len=1010, label3D_r.len=1010, label3D_r.len=1010
1010: data3D.len=1011, label3D_r.len=1011, label3D_r.len=1011
1011: data3D.len=1012, label3D_r.len=1012, label3D_r.len=1012
1012: data3D.len=1013, label3D_r.len=1013, label3D_r.len=1013
1013: data3D.len=1014, label3D_r.len=1014, label3D_r.len=1014
1014: data3D.len=1015, label3D_r.len=1015, label3D_r.len=1015
1015: data3D.len=1016, label3D_r.len=1016, label3D_r.len=1016
1016: data3D.len=1017, label3D_r.len=1017, label3D_r.len=1017
1017: da

1134: data3D.len=1135, label3D_r.len=1135, label3D_r.len=1135
1135: data3D.len=1136, label3D_r.len=1136, label3D_r.len=1136
1136: data3D.len=1137, label3D_r.len=1137, label3D_r.len=1137
1137: data3D.len=1138, label3D_r.len=1138, label3D_r.len=1138
1138: data3D.len=1139, label3D_r.len=1139, label3D_r.len=1139
1139: data3D.len=1140, label3D_r.len=1140, label3D_r.len=1140
1140: data3D.len=1141, label3D_r.len=1141, label3D_r.len=1141
1141: data3D.len=1142, label3D_r.len=1142, label3D_r.len=1142
1142: data3D.len=1143, label3D_r.len=1143, label3D_r.len=1143
1143: data3D.len=1144, label3D_r.len=1144, label3D_r.len=1144
1144: data3D.len=1145, label3D_r.len=1145, label3D_r.len=1145
1145: data3D.len=1146, label3D_r.len=1146, label3D_r.len=1146
1146: data3D.len=1147, label3D_r.len=1147, label3D_r.len=1147
1147: data3D.len=1148, label3D_r.len=1148, label3D_r.len=1148
1148: data3D.len=1149, label3D_r.len=1149, label3D_r.len=1149
1149: data3D.len=1150, label3D_r.len=1150, label3D_r.len=1150
1150: da

In [ ]:
if typ.lower() == 'keras':
    data3D = data3D.reshape(e+1,-1,feat_vec_len)
    label3D_r = label3D_r.reshape(e+1,-1,out_vec_len)
    label3D_i = label3D_i.reshape(e+1,-1,out_vec_len)

print("## data3D.len={0}, label3D_r.len={1}, label3D_r.len={2}".format(data3D.shape,label3D_r.shape,label3D_r.shape))

In [18]:
maxlen=185
isTRAINING = True
typ = 'keras'

feat_vec_len = 1230
out_vec_len = 963

if isTRAINING:
    data = opts.trData
    label_r = opts.trLabel_r
    label_i = opts.trLabel_i
    numframes = opts.trNumframes


data3D, label3D_r, label3D_i = None, None, None
numframes = np.cumsum(numframes)

#### WORKING ON IT ...
for e, frames in enumerate(numframes):
    frames= int(frames)
    pre_frames= int(numframes[e-1])

    # separate frames for each wavs
    d = data[:frames] if data3D is None else data[pre_frames:frames]
    r = label_r[:frames] if label3D_r is None else label_r[pre_frames:frames]
    i = label_i[:frames] if label3D_i is None else label_i[pre_frames:frames]


    # Add zeros add the end to make same sized
    if typ.lower() == 'keras':
        frame = d.shape[0]

        d = np.concatenate( (d, np.zeros(((maxlen-frame),feat_vec_len)) ), axis=0)
        r = np.concatenate( (r, np.zeros(((maxlen-frame),out_vec_len)) ), axis=0)
        i = np.concatenate( (i, np.zeros(((maxlen-frame),out_vec_len)) ), axis=0)

        # 3D for concat
        d = np.expand_dims(d, axis=0)
        r = np.expand_dims(r, axis=0)
        i = np.expand_dims(i, axis=0)
        

    data3D= d if data3D is None else np.concatenate( (data3D,d),axis=0 )
    label3D_r= r if label3D_r is None else np.concatenate( (label3D_r,r),axis=0 )
    label3D_i= i if label3D_i is None else np.concatenate( (label3D_i,i),axis=0 )

    if e%1000==1:
        print(e)

1
1001


KeyboardInterrupt: 

In [17]:
print("# data3D.len={0}, label3D_r.len={1}, label3D_r.len={2}".format(data3D.shape,label3D_r.shape,label3D_r.shape))

data3d: (1, 185, 1230)
label3D_r: (1, 185, 963)
label3D_i: (1, 185, 963)
data3d: (2, 185, 1230)
label3D_r: (2, 185, 963)
label3D_i: (2, 185, 963)
data3d: (3, 185, 1230)
label3D_r: (3, 185, 963)
label3D_i: (3, 185, 963)
data3d: (4, 185, 1230)
label3D_r: (4, 185, 963)
label3D_i: (4, 185, 963)
data3d: (5, 185, 1230)
label3D_r: (5, 185, 963)
label3D_i: (5, 185, 963)
data3d: (6, 185, 1230)
label3D_r: (6, 185, 963)
label3D_i: (6, 185, 963)
data3d: (7, 185, 1230)
label3D_r: (7, 185, 963)
label3D_i: (7, 185, 963)
data3d: (8, 185, 1230)
label3D_r: (8, 185, 963)
label3D_i: (8, 185, 963)
data3d: (9, 185, 1230)
label3D_r: (9, 185, 963)
label3D_i: (9, 185, 963)
data3d: (10, 185, 1230)
label3D_r: (10, 185, 963)
label3D_i: (10, 185, 963)
data3d: (11, 185, 1230)
label3D_r: (11, 185, 963)
label3D_i: (11, 185, 963)
data3d: (12, 185, 1230)
label3D_r: (12, 185, 963)
label3D_i: (12, 185, 963)
data3d: (13, 185, 1230)
label3D_r: (13, 185, 963)
label3D_i: (13, 185, 963)
data3d: (14, 185, 1230)
label3D_r: (14,

KeyboardInterrupt: 

In [15]:
len(opts.cvNumframes)

3300

In [46]:
data3D.shape
data3D.reshape(e+1,-1,feat_vec_len)

array([[[ 0.58509266,  0.16574308, -0.09612523, ...,  0.62136084,
          0.47224903,  0.6217047 ],
        [ 0.58509266,  0.16574308, -0.09612523, ...,  0.55170929,
          0.36697525,  0.24092968],
        [ 0.58509266,  0.16574308, -0.09612523, ...,  0.44091851,
          0.24544191, -0.09473364],
        ..., 
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.32243583,  0.46705359,  0.72176397, ...,  0.182722  ,
          0.18001671,  0.49387121],
        [ 0.08927108,  0.06295258,  0.09079958, ...,  0.14125155,
          0.17332885,  0.49526078],
        [-0.14226571, -0.02212001,  0.06352244, ...,  0.09782532,
          0.13277772,  0.3894659 ],
        ..., 
        [ 0.        ,  0.        ,  0.        , ...,

In [6]:
n= np.array([range(10),range(40,50),range(78,88)])
n

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
       [78, 79, 80, 81, 82, 83, 84, 85, 86, 87]])

In [16]:
n.flatten()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 40, 41, 42, 43, 44, 45, 46,
       47, 48, 49, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87])

In [12]:
m=n.reshape(3,5,-1)
m

array([[[ 0,  1],
        [ 2,  3],
        [ 4,  5],
        [ 6,  7],
        [ 8,  9]],

       [[40, 41],
        [42, 43],
        [44, 45],
        [46, 47],
        [48, 49]],

       [[78, 79],
        [80, 81],
        [82, 83],
        [84, 85],
        [86, 87]]])

In [8]:
Max_RNN = 185

feat_vec_len = 1230
out_vec_len = 963

### RNN 1
single bidirectional GRU layer

real+img (963+963)=1926-d output vector

In [16]:
model = Sequential()

model.add(Bidirectional(GRU(Max_RNN, return_sequences=True, stateful=True),batch_size=128, input_shape=(Max_RNN,feat_vec_len)))
# model.add(Dropout(0.2))
# model.add(Bidirectional(GRU(Max_RNN, return_sequences=True)))

model.add(Dense(out_vec_len*2, activation='linear'))

model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

for e in range(80):
    for (b_x,b_y), (v_x,v_y) in zip(next_batch_rnn(opts,128), next_batch_rnn(opts,128,isTrainCycle=False)):
        model.fit(b_x, b_y, validation_data=(v_x,v_y), shuffle=True, verbose=1,batch_size=128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_11 (Bidirectio (128, 185, 370)           1571760   
_________________________________________________________________
dense_5 (Dense)              (128, 185, 1926)          714546    
Total params: 2,286,306
Trainable params: 2,286,306
Non-trainable params: 0
_________________________________________________________________
None
Train on 128 samples, validate on 128 samples
Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[185,555]
	 [[Node: Variable_102/Assign = Assign[T=DT_FLOAT, _class=["loc:@Variable_102"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/gpu:0"](Variable_102, Const_227)]]

Caused by op 'Variable_102/Assign', defined at:
  File "/home/knayem/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/knayem/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/knayem/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/knayem/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/knayem/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/knayem/anaconda3/lib/python3.6/site-p

### Parameters

In [ ]:
## HAVE to use OPS PARAMS ***********
learning_rate = 0.001
# training_epochs = 80
# batch_size = 256
display_step = 1

### Network Parameters

In [ ]:
## HAVE to use NET_STRUCTURE ***********
n_input_dim = 195192.0
n_input = 1230  # data input
n_hidden_1 = 1024  # 1st layer number of neurons
n_hidden_2 = 1024  # 2nd layer number of neurons
n_hidden_3 = 1024  # 3rd layer number of neurons
n_classes = (963 + 963)  # total classes (real+imaginary)

# For checking
# train_time = np.zeros(training_epochs)

# validation_error = np.full((1), np.inf)
# min_validation_error = np.full((1), np.inf)

Best_Cost = - np.inf
Best_Weight, Best_Bias = None, None
Best_epoch = -1

# For displaying
PREVIOUS_10 = 10
DIFF_THRESHOLD = 0.000001


### Fully-Connected 3 layer Feed Forward Net

In [ ]:
X = tf.placeholder(tf.float32, shape=[None, n_input])
Y = tf.placeholder(tf.float32, shape=[None, n_classes])


# Xavier Initialization
def weight_variable(shape, stddev=None, stddev_2=None):
    if stddev is None:
        initial = tf.truncated_normal(shape, stddev=np.sqrt(2.0 / sum(shape)))
    elif stddev > 0.0:
        if stddev_2 is not None:
            r, c = shape
            initial1 = tf.truncated_normal([r, c//2], stddev=stddev)
            initial2 = tf.truncated_normal([r, c//2], stddev=stddev_2)
            initial = tf.concat([initial1,initial2], axis=1)
        else:
            initial = tf.truncated_normal(shape, stddev=stddev)

    else:
        initial = tf.constant(0.0, shape=shape)

    return tf.Variable(initial)


def bias_variable(shape, stddev=None, stddev_2=None):
    # initial = tf.constant(0.1, shape=shape)
    if stddev is None:
        initial = tf.truncated_normal(shape, stddev=np.sqrt(1.0 / sum(shape)))
    elif stddev > 0.0:
        if stddev_2 is not None:
            r, c = shape
            initial1 = tf.truncated_normal([r, c//2], stddev=stddev)
            initial2 = tf.truncated_normal([r, c//2], stddev=stddev_2)
            initial = tf.concat([initial1,initial2], axis=1)
        else:
            initial = tf.truncated_normal(shape, stddev=stddev)

    else:
        initial = tf.constant(0.0, shape=shape)

    return tf.Variable(initial)

In [ ]:
# old way
weights = {
    'h1': weight_variable([n_input, n_hidden_1],0.001),
    'h2': weight_variable([n_hidden_1, n_hidden_2],0.001),
    'h3': weight_variable([n_hidden_2, n_hidden_3],0.001),
    'out': weight_variable([n_hidden_3, n_classes],0.001,0.0)
}
biases = {
    'b1': bias_variable([1, n_hidden_1],0.0),
    'b2': bias_variable([1, n_hidden_2],0.0),
    'b3': bias_variable([1, n_hidden_3],0.0),
    'out': bias_variable([1, n_classes],0.001,0.0)
}

### Create model

In [ ]:
def multilayer_NN(x):
    layer_1 = tf.nn.relu(tf.matmul(x, weights['h1']) + biases['b1'])
    layer_2 = tf.nn.relu(tf.matmul(layer_1, weights['h2']) + biases['b2'])
    layer_3 = tf.nn.relu(tf.matmul(layer_2, weights['h3']) + biases['b3'])
    out_layer = tf.matmul(layer_3, weights['out']) + biases['out']

    return out_layer


def calc(x, y):
    # Returns predictions and error
    predictions = multilayer_NN(x)

    # Define loss and optimizer
    r, c = y.get_shape().as_list()
    r = opts.sgd_batch_size
    print('r:', type(r), r, ',c:', type(c), c)

    # TRAIN: matlab
    # cost1 = 0.5 * sum(sum((pred_real - label_real). ^ 2)) / num_sample;
    # cost2 = 0.5 * sum(sum((pred_imag - label_imag). ^ 2)) / num_sample;
    # cost = cost1 + cost2;
    cost1 = tf.reduce_sum(tf.squared_difference(y[:, :c//2], predictions[:, :c//2]))
    cost2 = tf.reduce_sum(tf.squared_difference(y[:, c//2:], predictions[:, c//2:]))
    loss_t = 0.5*(cost1+cost2)/r
    # mse = tf.losses.mean_squared_error(labels=y, predictions=predictions,weights=0.5)
    # loss_t = tf.divide(tf.reduce_sum(mse), r)


    # DEV: matlab
    # dev_perfs = -mean(sum((dev_label_real - dev_netout1). ^ 2)) - mean(sum((dev_label_imag - dev_netout2). ^ 2));
    mse_r = tf.reduce_sum(tf.squared_difference(y[:, :c // 2], predictions[:, :c // 2]), axis=0)
    mse_i = tf.reduce_sum(tf.squared_difference(y[:, c // 2:], predictions[:, c // 2:]), axis=0)

    loss_d = -tf.reduce_mean(mse_r)-tf.reduce_mean(mse_i)


    return [predictions, loss_t, mse_r, mse_i, loss_d ]

In [ ]:
def write_file(best_weights, best_biases, DNN_NET_FILE):
    W_1, W_2, W_3, W_4 = [np.array(best_weights['h1'], ndmin=2), np.array(best_weights['h2'], ndmin=2),
                          np.array(best_weights['h3'], ndmin=2), np.array([], ndmin=2)]
    W_1, W_2, W_3, W_4 = W_1.T, W_2.T, W_3.T, W_4

    print('W_1: ', W_1.shape, 'W_2: ', W_2.shape, 'W_3: ', W_3.shape, 'W_4: ', W_4.shape)

    b_1, b_2, b_3, b_4 = [np.array(best_biases['b1'], ndmin=2), np.array(best_biases['b2'], ndmin=2),
                          np.array(best_biases['b3'], ndmin=2), np.array([], ndmin=2)]
    b_1, b_2, b_3, b_4 = b_1.T, b_2.T, b_3.T, b_4

    print('b_1: ', b_1.shape, 'b_2: ', b_2.shape, 'b_3: ', b_3.shape, 'b_4: ', b_4.shape)

    Wo, bo = np.array(best_weights['out'], ndmin=2), np.array(best_biases['out'], ndmin=2)

    Wo1_1, Wo1_2, Wo1_3, Wo1_4 = [np.array([], ndmin=2), np.array([], ndmin=2), np.array([], ndmin=2), Wo[:, :963].T]
    bo1_1, bo1_2, bo1_3, bo1_4 = [np.array([], ndmin=2), np.array([], ndmin=2), np.array([], ndmin=2),
                                  bo[:, :963].T]
    # Wo1_1, Wo1_2, Wo1_3, Wo1_4 = [np.array([], ndmin=2), np.array([], ndmin=2), np.array([], ndmin=2), Wo[:963]]
    # bo1_1, bo1_2, bo1_3, bo1_4 = [np.array([],ndmin=2), np.array([],ndmin=2), np.array([],ndmin=2),
    #                               np.reshape(np.transpose(bo[:963]), (963, 1))]


    Wo2_1, Wo2_2, Wo2_3, Wo2_4 = [np.array([], ndmin=2), np.array([], ndmin=2), np.array([], ndmin=2), Wo[:, 963:].T]
    bo2_1, bo2_2, bo2_3, bo2_4 = [np.array([], ndmin=2), np.array([], ndmin=2), np.array([], ndmin=2),
                                  bo[:, 963:].T]
    # Wo2_1, Wo2_2, Wo2_3, Wo2_4 = [np.array([],ndmin=2), np.array([],ndmin=2), np.array([],ndmin=2), Wo[963:]]
    # bo2_1, bo2_2, bo2_3, bo2_4 = [np.array([],ndmin=2), np.array([],ndmin=2), np.array([],ndmin=2),
    #                               np.reshape(np.transpose(bo[963:]), (963, 1))]

    print('Wo1_1: ', Wo1_1.shape, 'Wo1_2: ', Wo1_2.shape, 'Wo1_3: ', Wo1_3.shape, 'Wo1_4: ', Wo1_4.shape)
    print('bo1_1: ', bo1_1.shape, 'bo1_2: ', bo1_2.shape, 'bo1_3: ', bo1_3.shape, 'bo1_4: ', bo1_4.shape)

    print('Wo2_1: ', Wo2_1.shape, 'Wo2_2: ', Wo2_2.shape, 'Wo2_3: ', Wo2_3.shape, 'Wo2_4: ', Wo2_4.shape)
    print('bo2_1: ', bo2_1.shape, 'bo2_2: ', bo2_2.shape, 'bo2_3: ', bo2_3.shape, 'bo2_4: ', bo2_4.shape)

    # Param_Dict = {'W': np.transpose([W_1,W_2, W_3]), 'b':np.transpose([b_1,b_2, b_3]) }
    # Param_Dict = {'W': { (W_1, W_2, (W_3)}, 'b': {(b_1), (b_2), (b_3)}}
    Param_Dict = np.core.records.fromarrays(
        [[W_1, W_2, W_3, W_4], [b_1, b_2, b_3, b_4], [Wo1_1, Wo1_2, Wo1_3, Wo1_4], [bo1_1, bo1_2, bo1_3, bo1_4],
         [Wo2_1, Wo2_2, Wo2_3, Wo2_4], [bo2_1, bo2_2, bo2_3, bo2_4]], names='W,b,Wo1,bo1,Wo2,bo2')

    # print(Param_Dict.shape, Param_Dict)
    master_dict = {'struct_net': [Param_Dict]}

    sio.savemat(DNN_NET_FILE, master_dict, format='5', long_field_names=True)

### main()

In [ ]:
# TRAIN ops
y_p, loss_op, m_r, m_i, _ = calc(X, Y)
train_op = tf.train.AdamOptimizer().minimize(loss=loss_op)


sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# Training cycle

for epoch in range(opts.sgd_max_epoch):

    s = time.time()
    cost_sum = 0.0

    ##### Create Train Batch, evaluate COST, Weight, Bias #####
    ###########################################################
    for batch_num, (batch_x, batch_y) in enumerate(opts.next_batch(opts.trData.shape[0], opts.sgd_batch_size)):

        _, c, epoch_w, epoch_b  = sess.run([train_op, loss_op, weights, biases ],
                                          feed_dict={X: batch_x, Y: batch_y})
        cost_sum += c

        if batch_num % 1000 == 0:
            print('[T] - Epoch:', epoch, ', batch_num:', batch_num, ", Cost:", c, "Cost Sum:", cost_sum)

    print('[T] - Epoch:', epoch, ",Sum:", cost_sum)


    ################ Validation in whole batch ################
    ###########################################################

    avg_cost, sum_mse_r, sum_mse_i = 0.0, np.zeros(n_classes//2), np.zeros(n_classes//2)
    ##### Create DEV Batch, evaluate COST, Weight, Bias #####
    ###########################################################
    for batch_num, (batch_x, batch_y) in enumerate(opts.next_batch(opts.cvData.shape[0], opts.sgd_batch_size, isTrainCycle=False)):

        mse_r, mse_i = sess.run([m_r, m_i], feed_dict={X: batch_x, Y: batch_y})
        # print(sum_mse_r.shape, mse_r.shape, mse_i.shape)
        sum_mse_r += (mse_r)
        sum_mse_i += (mse_i)

    avg_cost = - np.mean(sum_mse_r) - np.mean(sum_mse_i)
    # print('[D] - Epoch:', epoch, ", Mean Real:", - np.mean(sum_mse_r), ", Mean Img:" , - np.mean(sum_mse_i), ", Avg Cost:", avg_cost)

    print("[D] - Epoch:{0}, Avg Cost:{1:.5f}, Real(mean,var):({2:.5f},{3:.5f}), Img(mean,var):({4:.5f},{5:.5f})".format( epoch,avg_cost, -np.mean(sum_mse_r),-np.var(sum_mse_r), -np.mean(sum_mse_i),
          -np.var(sum_mse_i)) )

    ####### Min validation error, update weights, bias #########
    ############################################################


    if avg_cost > Best_Cost:
        Best_Cost = avg_cost
        Best_Weight = epoch_w
        Best_Bias = epoch_b
        Best_epoch = epoch

        print('***** [D] - Best Model at Epoch:', epoch, ", Avg Cost:", avg_cost, '*****')


    ######################  Write Model File ###################
    ############################################################

    print('[] - Elapsed Time: {0:.2f}(s), {1:.2f}(min)'.format( time.time()-s, (time.time()-s)/60 ) )

In [ ]:
# Training DONE !!!
print("Optimization Finished!")
print('***** [-] - Best Model at Epoch:', Best_epoch, ", Best Val Cost:", Best_Cost, '*****')
write_file(Best_Weight, Best_Bias, DNN_NET_FILE)
print(" File Write complete")
